In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

In [89]:
# Reading input

# articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
# customers = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
sample_submission = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')\
[['customer_id']]

In [3]:
transactions_train['customer_id'].nunique(), sample_submission.shape[0]

(1362281, 1371980)

In [4]:
pd.merge(transactions_train['customer_id'].drop_duplicates(),
         sample_submission['customer_id'],
         left_on = 'customer_id',
         right_on = 'customer_id'
        ).shape

(1362281, 1)

In [5]:
transactions_train['t_dat_datetime']  = pd.to_datetime(transactions_train['t_dat'])

transactions_train['t_month'] = transactions_train['t_dat_datetime'].dt.month

transactions_train['t_year']  = transactions_train['t_dat_datetime'].dt.year

In [6]:
pd.merge(transactions_train[(transactions_train['t_month'].between(9,10, inclusive='both'))\
                           ]['customer_id'].drop_duplicates(),
         sample_submission['customer_id'],
         left_on = 'customer_id',
         right_on = 'customer_id'
        ).shape

(655025, 1)

In [7]:
transactions_train['max_purc_date'] = transactions_train.groupby('customer_id')['t_dat_datetime'].transform('max')

transactions_train['min_purc_date'] = transactions_train.groupby('customer_id')['t_dat_datetime'].transform('min')

transactions_train['total_visits'] = transactions_train.groupby('customer_id')['t_dat'].transform('nunique')

transactions_train['total_item_bought'] = transactions_train.groupby('customer_id')['article_id'].transform('nunique')

transactions_train['max_year'] = transactions_train.groupby('customer_id')['t_year'].transform('max')

In [12]:
# transactions_train_filt = transactions_train[transactions_train['max_year'] == 2020]

# transactions_train_filt['product_reach'] = transactions_train_filt.groupby('article_id')['customer_id'].transform('nunique')

# transactions_train_filt = transactions_train_filt[transactions_train_filt['product_reach'] > 8]

transactions_train['repeat_purchase'] = transactions_train.groupby(['customer_id','article_id'])['t_dat'].transform('nunique')

transactions_train_filt = transactions_train[transactions_train['repeat_purchase'] > 1]

# transactions_train_filt = transactions_train_filt[transactions_train_filt['total_visits'] >= 2]

# transactions_train_filt = transactions_train_filt[transactions_train_filt['total_item_bought'] >= 12]

In [13]:
transactions_train_filt.shape

(2856746, 14)

In [14]:
product_purchase_cycle = transactions_train_filt[['customer_id','article_id','t_dat_datetime']].\
sort_values(['customer_id','article_id','t_dat_datetime']).drop_duplicates()

In [15]:
product_purchase_cycle['t_dat_datetime_lag'] = product_purchase_cycle.\
groupby(['customer_id','article_id'])['t_dat_datetime'].shift(1)

In [16]:
product_purchase_cycle['purchase_cycle'] = product_purchase_cycle['t_dat_datetime'] - product_purchase_cycle['t_dat_datetime_lag']

In [17]:
Average_Purchase_Cycle = product_purchase_cycle[~(product_purchase_cycle['purchase_cycle'].isnull())].\
groupby(['customer_id','article_id']).agg({"purchase_cycle":"median"}).reset_index()

In [18]:
transactions_train_filt = pd.merge(transactions_train_filt,
                                   Average_Purchase_Cycle,
                                   left_on = ['customer_id','article_id'],
                                   right_on = ['customer_id','article_id'],
                                   how='inner'
                                  )

In [19]:
transactions_train_filt.shape

(2856746, 15)

In [20]:
transactions_train_filt['product_last_purchase_date'] = transactions_train_filt.\
groupby(['customer_id','article_id'])['t_dat_datetime'].transform("max")

In [21]:
from datetime import datetime
from datetime import timedelta

In [22]:
transactions_train_filt['purchase_cycle_days'] = transactions_train_filt['purchase_cycle'].astype('str').str.split(' ', expand=True)[0]

In [23]:
transactions_train_filt['next_purchase_date'] = transactions_train_filt['product_last_purchase_date'] + transactions_train_filt['purchase_cycle_days'].astype(int).map(timedelta)

In [24]:
start_date = transactions_train_filt['t_dat_datetime'].max() + timedelta(-90)
end_date = transactions_train_filt['t_dat_datetime'].max() + timedelta(90)

In [25]:
datetime.date(transactions_train_filt['t_dat_datetime'].max()) 

datetime.date(2020, 9, 22)

In [26]:
start_date, end_date

(Timestamp('2020-06-24 00:00:00'), Timestamp('2020-12-21 00:00:00'))

In [27]:
len(transactions_train_filt['customer_id'].unique())

377478

In [58]:
customer_next_purchase = transactions_train_filt[transactions_train_filt['next_purchase_date'].\
                        between(start_date, end_date, inclusive="both")]\
[['customer_id','t_dat','article_id']].drop_duplicates()

In [59]:
customer_next_purchase['repeat_purchase'] = customer_next_purchase.groupby(['customer_id'])['article_id'].transform('count')

In [60]:
customer_next_purchase['product_imp_rank'] = customer_next_purchase.groupby(['customer_id'])['repeat_purchase'].rank(method="first",ascending=True)

In [61]:
customer_next_purchase['max_rank'] = customer_next_purchase[customer_next_purchase['product_imp_rank'] <= 12].\
groupby('customer_id')['product_imp_rank'].transform('max')

In [62]:
customer_next_purchase = customer_next_purchase[['customer_id','article_id','max_rank']]

customer_next_purchase = customer_next_purchase.sort_values(['customer_id','article_id'])

In [63]:
# Filling Products 

top_12_products = customer_next_purchase.groupby(['article_id']).agg(cust_count=('customer_id','nunique')).reset_index()

top_12_products = top_12_products.sort_values('cust_count', ascending = False).head(12)[['article_id']]

In [65]:
len(customer_next_purchase['customer_id'].unique())

117328

In [66]:
filler_cust_prod_comb= pd.merge(customer_next_purchase['customer_id'].drop_duplicates(), 
top_12_products['article_id'], how = 'cross')

In [67]:
customer_next_purchase= pd.concat([customer_next_purchase[['customer_id', 'article_id']],
          filler_cust_prod_comb[['customer_id', 'article_id']]],
          axis = 0
         ).drop_duplicates()

In [68]:
customer_next_purchase['row_num'] = customer_next_purchase.groupby(['customer_id']).cumcount()

In [72]:
customer_next_purchase = customer_next_purchase[customer_next_purchase['row_num'] <= 11]

In [73]:
customer_next_purchase['article_id'] = ' 0'+customer_next_purchase['article_id'].astype('str')

In [82]:
customer_next_purchase = customer_next_purchase.groupby(['customer_id'])['article_id'].sum().reset_index()

In [77]:
top_12_products['article_id'] = ' 0'+top_12_products['article_id'].astype('str')

In [79]:
top_12_products_for_missin_cust = top_12_products['article_id'].sum()
top_12_products_for_missin_cust

' 0706016001 0706016002 0372860001 0464297007 0160442007 0751471001 0399223001 0372860002 0156231001 0608776002 0448509014 0610776002'

In [90]:
sample_submission = pd.merge(sample_submission,
                             customer_next_purchase,
                             left_on = ['customer_id'],
                             right_on = ['customer_id'],
                             how = 'left'
                            ).fillna(top_12_products_for_missin_cust)

In [ ]:
# no_products_cust['article_id'].fillna( "0706016001 0706016002 0372860001 0399223001 0464297007 0160442007 0610776002 0156231001 0399256001 0372860002 0706016003 0759871002", inplace=True)
# no_products_cust.shape

In [94]:
sample_submission.rename(columns={'article_id':'prediction'}, inplace=True)
sample_submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0464297007 0...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0351484002 0723529001 0706016001 0706016002 0...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0464297007 0...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0464297007 0...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0464297007 0...


In [95]:
sample_submission.to_csv('submission.csv', index =False)